In [ ]:
from duel_game.game import DuelGame
from duel_game.player import Player
from duel_game.data_processor import Tracker
from duel_game.dataset_repo import DatasetRepository


from dotenv import load_dotenv
import json
import hashlib
import random

In [ ]:
def play_game_and_store_samples(player_1, player_2, max_turns, seed):
    tracker = Tracker()
    game = DuelGame(player_1, player_2, tracker, max_turns)
    game.play_game()
    
    created_samples = tracker.get_samples()


def start_run():
    # instantiate a DatasetRepository class
    data_repo = DatasetRepository() 

    # ask the user to whether load an existing config template from database using its ID or load configurations from environment variables(.env)
        # load the data-generation-specific configuration from .env using a function 
            # hash the config and check config duplication using a method of DatasetRepository class - return id of the config template if any found, else 0
                # then ask the user to input an optional label and description for this config
                # get a new version number
                # send config, version number and label and description to RepositoryObject to save the config in database
            # use the retrieved config 

        # or load the config template from database using entered id  
    
    # input optional note for run
    # create run record using template id and note

    print("Dataset Run Initialization")
    print("1) Load config from environment (.env)")
    print("2) Load existing config template from database")

    choice = input("Select option (1 or 2): ").strip()

    # ----------------------------
    # LOAD CONFIG FROM ENVIRONMENT
    # ----------------------------
    if choice == "1":
        # load the data-generation-specific configuration from .env
        config = load_env_config()  # expected to return dict

        # hash the config deterministically
        config_json = json.dumps(config, sort_keys=True)
        config_hash = hashlib.sha256(config_json.encode()).hexdigest()

        # check config duplication
        existing_template_id = data_repo.is_config_available(config_hash)

        if existing_template_id:
            print(f"Existing config template found (ID={existing_template_id})")
            template_id = existing_template_id

        else:
            print("No existing config found. Creating new template.")

            label = input("Optional config label (press Enter to skip): ").strip()
            description = input("Optional config description (press Enter to skip): ").strip()

            version_number = data_repo.get_new_template_version_number()

            template = {
                "version": version_number,
                "label": label,
                "description": description,
                "config": config,
                "config_hash": config_hash,
            }

            template_id = data_repo.create_config_template(template)

    # --------------------------------
    # LOAD CONFIG TEMPLATE FROM DB
    # --------------------------------
    elif choice == "2":
        while True:
            template_id = int(input("Enter config template ID: ").strip())

            try:
                config = data_repo.get_config_template(template_id)
                print(f"Loaded config template {template_id}")
                break
            except ValueError as e:
                print(f"Invalid template ID: {template_id}")
                continue

    else:
        raise ValueError("Invalid selection. Must be 1 or 2.")

    # ----------------------------
    # CREATE DATASET RUN
    # ----------------------------
    run_note = input("Optional note for this run (press Enter to skip): ").strip()

    seed_input = input(
        "Enter random seed for this run (press Enter to auto-generate): "
    ).strip()

    if seed_input:
        seed = int(seed_input)
    else:
        seed = random.randint(0, 2**32 - 1)

    run_id = data_repo.create_run(
        template_id=template_id,
        note=run_note,
        seed=seed
    )

    print(
        f"Run created successfully "
        f"(run_id={run_id}, template_id={template_id}, seed={seed})"
    )

    # prepare run-specific configuration and play games, count samples and play as much as game needed to reach specified number of samples

    pass

def load_env_config():
    pass